In [3]:
!pip install --upgrade setuptools
!pip install --upgrade pip
!pip install  google-cloud-bigquery
!pip install pydata-google-auth
!pip install db-dtypes

Requirement already up-to-date: setuptools in /Users/user/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages (68.0.0)
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 23.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
    100% |████████████████████████████████| 2.1MB 7.6MB/s eta 0:00:01
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1


  Obtaining dependency information for google-cloud-bigquery from https://files.pythonhosted.org/packages/22/03/a5d04741507156b88e9101e91d97bd395d043e9de280b94b1d00cb0f4e03/google_cloud_bigquery-3.12.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for grpcio<2.0dev,>=1.47.0 from https://files.pythonhosted.org/packages/b0/da/329d6a84c67910b4c58141b01dcfb93536db56543452141c79b265da2981/grpcio-1.59.0-cp37-cp37m-macosx_10_10_universal2.whl.metadata
  Obtaining dependency information for google-api-core[grpc]!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.0,<3.0.0dev,>=1.31.5 from https://files.pythonhosted.org/packages/4d/ce/4fd62ea66b3508debc795e475336ce915929765870f0ad52328426ba016e/google_api_core-2.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for proto-plus<2.0.0dev,>=1.15.0 from https://files.pythonhosted.org/packages/36/5b/e02636d221917d6fa2a61289b3f16002eb4c93d51c0191ac8e896d527182/proto_plus-1.22.3-py3-none-any.whl.metadata
  Obtaining dependency information f

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 304.7 kB/s eta 0:00:00a 0:00:01
Using cached google_api_core-2.12.0-py3-none-any.whl (121 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 7.0 MB/s eta 0:00:00
DEPRECATION: pandas 0.23.4 has a non-standard dependency specifier pytz>=2011k. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pandas or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: packaging
    Found existing installation: packaging 17.1
    Uninstalling packaging-17.1:
      Successfully uninstalled packaging-17.1
  Attempting uninstall: requests
    Found existing installation: requests 2.19.1
    Uninstalling requests-2.19.1:
      Successfully uninstalled requests-2.19.1
DEPRECATION: pandas 0.23.4 has a non-standard dependency specifier pytz>=2011k. pip

In [1]:

import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)

In [46]:
import json
from google.cloud import bigquery 

variables = open('variables.json' , 'r')
variables = json.load(variables)

client = bigquery.Client(project=variables['bq_project_name'], credentials=credentials)
query = 'SELECT * FROM '+variables['bq_table_name']+' WHERE DATE(created_at, "Japan") BETWEEN "2021-11-22" AND "2022-11-21"'
df_tmp = client.query(query).to_dataframe()
df = df_tmp

SELECT * FROM tmp_hira.tweets_nijiholo WHERE DATE(created_at, "Japan") BETWEEN "2021-11-22" AND "2022-11-21"


In [19]:
df.head()

,quoted_id,quoted_status_id_str,in_reply_to_status_id,quote_count,timestamp_ms,possibly_sensitive,favorited,user_id,created_at,id_str,...,quoted_status_id,in_reply_to_user_id,retweeted_id,lang,retweet_count,in_reply_to_user_id_str,text,hololive_flg,nijisanji_flg,ix
0,<NA>,None,<NA>,0,1515669380658,<NA>,False,220312298,2018-01-11 11:16:20+00:00,951412456429043712,...,<NA>,<NA>,<NA>,ja,0,None,ときのそら生配信やってますよ～\n#KizunaAI #ミライアカリ #輝夜月 #SiroA...,True,False,1
1,<NA>,None,<NA>,0,1518876215659,False,False,3816372614,2018-02-17 14:03:35+00:00,964862897301069824,...,<NA>,<NA>,<NA>,ja,0,None,少し遅れてしまいましたがバレンタインそらちゃんを・・・首のもこもこはフードなのかな？という妄...,True,False,18
2,<NA>,None,<NA>,0,1521168575657,<NA>,False,2904280280,2018-03-16 02:49:35+00:00,974477752010076160,...,<NA>,<NA>,<NA>,ja,0,None,でろーんとみとみとの身長差イラスト、でろーんがみとみとを抱きしめてるのが多い。\nみとみとが...,False,True,46
3,<NA>,None,<NA>,0,1521353789666,True,False,974630611187937281,2018-03-18 06:16:29+00:00,975254595868831745,...,<NA>,<NA>,<NA>,ja,0,None,#森中びじゅつかん #森中花咲 \n花咲ちゃんの怖がってる声がとっても可愛いですね。きゅん...,False,True,51
4,<NA>,None,<NA>,0,1521631145663,False,False,1473977725,2018-03-21 11:19:05+00:00,976417911236538371,...,<NA>,<NA>,<NA>,ja,0,None,委員長すき(挨拶)\n#みとあーと https://t.co/jh4LIoZ4uO,False,True,73


In [13]:
import pandas as pd

df_tag_holo = pd.read_csv('tag_holo.csv')
df_tag_niji = pd.read_csv('tag_niji.csv')
df_tag_niji.head()

,name,tag,Twitter
0,月ノ美兎,#みとあーと,@MitoTsukino
1,勇気ちひろ,#ちーあーと,@Chihiro_yuki23
2,える,#えるの絵だよ,@Elu_World
3,樋口楓,#でろあーと,@HiguchiKaede
4,静凛,#凛Art,@ShizuRin23


In [47]:
for hako in ['niji', 'holo']:
    flg_name = 'nijisanji_flg' if hako == 'niji' else 'hololive_flg'
    df_tag = df_tag_niji if hako == 'niji' else df_tag_holo
    
    df_tagged = df[df[flg_name]==True][['user_id', 'created_at','text']]

    for _, row in df_tag.iterrows():
        df_tagged[row['name']] = df_tagged['text'].str.contains(row['tag'])
        
    df_tagged.to_csv('tweets_tagged_'+hako+'.csv')
        
    if hako == 'niji':
        df_niji = df_tagged
    else:
        df_holo = df_tagged

In [41]:
df_niji.iloc[:, 3:].sum().reset_index().rename(columns={0:'cnt'}).sort_values('cnt', ascending=False).head(10)

,index,cnt
107,甲斐田晴,711
16,剣持刀也,528
81,黛灰,317
18,叶,296
96,不破湊,288
70,三枝明那,272
119,壱百満天原サロメ,216
105,長尾景,189
14,伏見ガク,171
114,アクシア・クローネ,134


In [42]:
df_holo.iloc[:, 3:].sum().reset_index().rename(columns={0:'cnt'}).sort_values('cnt', ascending=False).head(10)
#test

,index,cnt
36,沙花叉クロヱ,227
3,星街すいせい,218
18,兎田ぺこら,195
10,湊あくあ,152
28,雪花ラミィ,150
14,大空スバル,142
37,風真いろは,137
23,天音かなた,134
15,大神ミオ,89
12,百鬼あやめ,85
